In [241]:
import numpy as np
import math
import scipy.stats as st
import plotly.express as px

In [242]:
def simulador(simulation_total_time, mean_arrival_time, mean_departure_time):
    #print("\nTempo de chegada | Tipo de evento | Número de amostras\n")
    n, number_of_arrivals, waiting_line_time, last_departure, service_time = 0, 0, 0, 0, 0
    queue, arrivals, event = [], [], [0,""]
    queue.append([np.random.exponential(scale=1/mean_arrival_time), "Chegada"])
    while(event[0] < simulation_total_time):
        event = queue.pop(0)
        if event[1] == "Chegada": 
            #print(f"{event[0]:8.4f} \t\t{'Chegada':14} {n:2} => {n+1:2}")
            queue.append([event[0] + np.random.exponential(scale=1/mean_arrival_time), "Chegada"])
            n, number_of_arrivals = n + 1, number_of_arrivals + 1
            if (n == 1):
                queue.append([event[0] + np.random.exponential(scale=1/mean_departure_time), "Saída"])
            if (n > 1):
                arrivals.append(event[0])
        else:
            if (len(arrivals) > 0):
                waiting_line_time += event[0] - arrivals.pop(0)
            #print(f"{event[0]:8.4f} \t\t{'Saída':14} {n:2} => {n-1:2}")
            n, service_time, last_departure = n - 1, service_time + (event[0] - last_departure), event[0] 
            if (n > 0):
                queue.append([event[0] + np.random.exponential(scale=1/mean_departure_time), "Saída"])
        queue.sort()
    return n, event[0], waiting_line_time/number_of_arrivals, service_time/number_of_arrivals, number_of_arrivals

In [243]:
total_simulation_time = 100000
mean_arrival_time = 1
mean_departure_time = 2

In [244]:
n, simul_time, mean_waiting_inline_time, mean_service_time, number_of_arrivals = simulador(total_simulation_time, mean_arrival_time, mean_departure_time)

In [245]:
print(f"\nTotal de pessoas que foram ao banco na simulação: {number_of_arrivals}")
print(f"Número de pessoas na fila ao final da simulação: {n}")
print(f"Tempo total da simulação: {simul_time:.4f}")
print(f"Tempo médio de espera na fila: {(mean_waiting_inline_time):.4f}")
print(f"Tempo médio de serviço: {(mean_service_time):.4f}")


Total de pessoas que foram ao banco na simulação: 99689
Número de pessoas na fila ao final da simulação: 1
Tempo total da simulação: 100000.2163
Tempo médio de espera na fila: 0.5103
Tempo médio de serviço: 1.0031


In [246]:
def simulator_confidence_interval(sample_size, confidence_level):
    """
    Simula o processo de fila m/m/1 um intervalo de confiança para tempo médio de espera na fila.

    sample_size: tamanho da amostra para a simulação
    confidence_level: nível de confiança desejado para o intervalo de confiança

    Retorna um intervalo de confiança para o tempo médio de espera na fila.
    """
    results = [simulador(1000, 1, 2)[2] for _ in range(sample_size)]


    # Cálculo do intervalo de confiança
    # proportion = np.sum(results) / sample_size
    # std_error = math.sqrt(proportion * (1 - proportion) / sample_size)
    # margin_error = std_error * st.norm.ppf(confidence_level)
    # lower_bound = proportion - margin_error #limite superior do IC
    # upper_bound = proportion + margin_error  # limite inferior do IC


    # Outro calculo
    x_barra = np.sum(results) / sample_size
    z = 1.96
    s = np.std(results)
    limite_inferior = x_barra - z*s/math.sqrt(sample_size)
    limite_superior = x_barra + z*s/math.sqrt(sample_size)

    # print(limite_inferior, limite_superior)
    return (limite_inferior, limite_superior), results
    return (lower_bound, upper_bound), results

intervalo, data = simulator_confidence_interval(1000, 0.95)

print(intervalo)

(0.49265554469717937, 0.5031855446244586)


In [247]:
px.histogram(data, nbins=50)

### Avaliando Resultados

Link para referência: https://people.revoledu.com/kardi/tutorial/Queuing/MM1-Queuing-System.html

Cálculo para 